In [1]:
import org.apache.spark.sql.functions.{udf, input_file_name};

// All column name strings defined beforehand, for uniformity and ease of editting
val pop = "Population"
val per_male = "Male"
val per_female = "Female"
val per_under_5 = "Under_5"
val per_5_9 = "5_9"
val per_10_14 = "10_14"
val per_15_19 = "15_19"
val per_20_24 = "20_24"
val per_25_34 = "25_34"
val per_35_44 = "35_44"
val per_45_54 = "45_54"
val per_55_59 = "55_59"
val per_60_64 = "60_64"
val per_65_74 = "65_74"
val per_75_84 = "75_84"
val per_85_plus = "85plus"

val per_one_race = "One_Race"
val per_only_white = "Only_White"
val per_only_black = "Only_Black"
val per_only_native = "Only_Native"
val per_only_cherokee = "Only_Cherokee"
val per_only_chippewa = "Only_Chippewa"
val per_only_navajo = "Only_Navajo"
val per_only_sioux = "Only_Sioux"
val per_only_asian = "Only_Asian"
val per_only_indian = "Only_Indian"
val per_only_chinese = "Only_Chinese"
val per_only_filipino = "Only_Filipino"
val per_only_japanese = "Only_Japanese"
val per_only_korean = "Only_Korean"
val per_only_vietnamese = "Only_Vietnamese"
val per_only_other_asian = "Only_Asian_Other"
val per_only_pacific_islander = "Only_Pacific"
val per_only_hawaiian = "Only_Hawaiian"
val per_only_guam = "Only_Guamanian"
val per_only_samoan = "Only_Samoan"
val per_only_pacific_other = "Only_Pacific_Other"
val per_only_other = "Only_Other"
val per_two_races = "Twoplus_Races"
val per_white_black = "White_Black"
val per_white_native = "White_Native"
val per_white_asian = "White_Asian"
val per_black_native = "Black_Native"

val per_white = "White"
val per_black = "Black"
val per_native = "Native"
val per_asian = "Asian"
val per_pacific = "Pacific"
val per_other = "Other"

val per_hispanic = "Hispanic"
val per_mexican = "Mexican"
val per_puerto_rican = "Puerto_Rican"
val per_cuban = "Cuban"
val per_hispanic_other = "Hispanic_Other"
val per_not_hispanic = "Not_Hispanic"

val extractYear = udf((filename: String) => filename.split("/").last.substring(7,11))

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
19,application_1598467819421_0020,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.functions.{udf, input_file_name}
pop: String = Population
per_male: String = Male
per_female: String = Female
per_under_5: String = Under_5
per_5_9: String = 5_9
per_10_14: String = 10_14
per_15_19: String = 15_19
per_20_24: String = 20_24
per_25_34: String = 25_34
per_35_44: String = 35_44
per_45_54: String = 45_54
per_55_59: String = 55_59
per_60_64: String = 60_64
per_65_74: String = 65_74
per_75_84: String = 75_84
per_85_plus: String = 85plus
per_one_race: String = One_Race
per_only_white: String = Only_White
per_only_black: String = Only_Black
per_only_native: String = Only_Native
per_only_cherokee: String = Only_Cherokee
per_only_chippewa: String = Only_Chippewa
per_only_navajo: String = Only_Navajo
per_only_sioux: String = Only_Sioux
per_only_asian: String = Only_Asian
per_only_indian: String = Only_Indian
per_only_chinese: String = Only_Chinese
per_only_filipino: String = Only_Filipino
per_only_japanese: String = Only_Japanese
per_only_korean: String

In [2]:
var first: Boolean = true
var genAge: org.apache.spark.sql.DataFrame = null
var complexRace: org.apache.spark.sql.DataFrame = null
var simpleRace: org.apache.spark.sql.DataFrame = null
var hispanic: org.apache.spark.sql.DataFrame = null

for (i <- 2011 to 2016) {
    val rawDemIntermediate = spark.read.option("header", "true").
        csv(s"s3://agimodeltrainer/DATA/DEM/ACSDP5Y${i}.csv").
        filter($"GEO_ID" =!= "id"). // Removes 2nd header line
        withColumn("FIPS", substring($"GEO_ID", 10, 5)).
        withColumn("Year", extractYear(input_file_name))

    rawDemIntermediate.cache

    val genAgeIntermediate = rawDemIntermediate.select(
            $"FIPS",
            $"Year".cast("Int"),
            $"DP05_0001E".as(pop).cast("Int"),
            $"DP05_0002PE".as(per_male).cast("Double"),
            $"DP05_0003PE".as(per_female).cast("Double"),
            $"DP05_0004PE".as(per_under_5).cast("Double"),
            $"DP05_0005PE".as(per_5_9).cast("Double"),
            $"DP05_0006PE".as(per_10_14).cast("Double"),
            $"DP05_0007PE".as(per_15_19).cast("Double"),
            $"DP05_0008PE".as(per_20_24).cast("Double"),
            $"DP05_0009PE".as(per_25_34).cast("Double"),
            $"DP05_0010PE".as(per_35_44).cast("Double"),
            $"DP05_0011PE".as(per_45_54).cast("Double"),
            $"DP05_0012PE".as(per_55_59).cast("Double"),
            $"DP05_0013PE".as(per_60_64).cast("Double"),
            $"DP05_0014PE".as(per_65_74).cast("Double"),
            $"DP05_0015PE".as(per_75_84).cast("Double"),
            $"DP05_0016PE".as(per_85_plus).cast("Double")
    )

    val complexRaceIntermediate = rawDemIntermediate.select(
            $"FIPS",
            $"Year".cast("Int"),
            $"DP05_0031PE".as(per_one_race).cast("Double"),
            $"DP05_0032PE".as(per_only_white).cast("Double"),
            $"DP05_0033PE".as(per_only_black).cast("Double"),
            $"DP05_0034PE".as(per_only_native).cast("Double"),
            $"DP05_0035PE".as(per_only_cherokee).cast("Double"),
            $"DP05_0036PE".as(per_only_chippewa).cast("Double"),
            $"DP05_0037PE".as(per_only_navajo).cast("Double"),
            $"DP05_0038PE".as(per_only_sioux).cast("Double"),
            $"DP05_0039PE".as(per_only_asian).cast("Double"),
            $"DP05_0040PE".as(per_only_indian).cast("Double"),
            $"DP05_0041PE".as(per_only_chinese).cast("Double"),
            $"DP05_0042PE".as(per_only_filipino).cast("Double"),
            $"DP05_0043PE".as(per_only_japanese).cast("Double"),
            $"DP05_0044PE".as(per_only_korean).cast("Double"),
            $"DP05_0045PE".as(per_only_vietnamese).cast("Double"),
            $"DP05_0046PE".as(per_only_other_asian).cast("Double"),
            $"DP05_0047PE".as(per_only_pacific_islander).cast("Double"),
            $"DP05_0048PE".as(per_only_hawaiian).cast("Double"),
            $"DP05_0049PE".as(per_only_guam).cast("Double"),
            $"DP05_0050PE".as(per_only_samoan).cast("Double"),
            $"DP05_0051PE".as(per_only_pacific_other).cast("Double"),
            $"DP05_0052PE".as(per_only_other).cast("Double"),
            $"DP05_0053PE".as(per_two_races).cast("Double"),
            $"DP05_0054PE".as(per_white_black).cast("Double"),
            $"DP05_0055PE".as(per_white_native).cast("Double"),
            $"DP05_0056PE".as(per_white_asian).cast("Double"),
            $"DP05_0057PE".as(per_black_native).cast("Double")
    )

    val simpleRaceIntermediate = rawDemIntermediate.select(
            $"FIPS",
            $"Year".cast("Int"),
            $"DP05_0059PE".as(per_white).cast("Double"),
            $"DP05_0060PE".as(per_black).cast("Double"),
            $"DP05_0061PE".as(per_native).cast("Double"),
            $"DP05_0062PE".as(per_asian).cast("Double"),
            $"DP05_0063PE".as(per_pacific).cast("Double"),
            $"DP05_0064PE".as(per_other).cast("Double")
    )

    val hispanicIntermediate = rawDemIntermediate.select(
            $"FIPS",
            $"Year".cast("Int"),
            $"DP05_0066PE".as(per_hispanic).cast("Double"),
            $"DP05_0067PE".as(per_mexican).cast("Double"),
            $"DP05_0068PE".as(per_puerto_rican).cast("Double"),
            $"DP05_0069PE".as(per_cuban).cast("Double"),
            $"DP05_0070PE".as(per_hispanic_other).cast("Double"),
            $"DP05_0071PE".as(per_not_hispanic).cast("Double")
    )
    
    if (first) {
        first = false
        genAge = genAgeIntermediate
        complexRace = complexRaceIntermediate
        simpleRace = simpleRaceIntermediate
        hispanic = hispanicIntermediate
    } else {
        genAge = genAge.union(genAgeIntermediate)
        complexRace = complexRace.union(complexRaceIntermediate)
        simpleRace = simpleRace.union(simpleRaceIntermediate)
        hispanic = hispanic.union(hispanicIntermediate)
    }
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

first: Boolean = true
genAge: org.apache.spark.sql.DataFrame = null
complexRace: org.apache.spark.sql.DataFrame = null
simpleRace: org.apache.spark.sql.DataFrame = null
hispanic: org.apache.spark.sql.DataFrame = null


In [3]:
// Test to make sure counts of all tables are the same, and no null values
val tables = List(genAge, complexRace, simpleRace, hispanic)

if (tables.exists(_.count != tables.head.count) || tables.exists(t => { t.count != t.na.drop.count } )) {
    println("Test failed")
} else { 
    println("Test passed")
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

tables: List[org.apache.spark.sql.DataFrame] = List([FIPS: string, Year: int ... 16 more fields], [FIPS: string, Year: int ... 27 more fields], [FIPS: string, Year: int ... 6 more fields], [FIPS: string, Year: int ... 6 more fields])
Test passed


In [4]:
// >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> Schema change from 2011-2016!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
// And we start the data cleaning dance all over again....
val rawDem17 = spark.read.option("header", "true").
    csv("s3://agimodeltrainer/DATA/DEM/ACSDP5Y2017.csv").
    filter($"GEO_ID" =!= "id"). // Removes 2nd header line
    withColumn("FIPS", substring($"GEO_ID", 10, 5)).
    withColumn("Year", extractYear(input_file_name))

rawDem17.cache

val genAge17 = rawDem17.select(
        $"FIPS",
        $"Year".cast("Int"),
        $"DP05_0001E".as(pop).cast("Int"),
        $"DP05_0002PE".as(per_male).cast("Double"),
        $"DP05_0003PE".as(per_female).cast("Double"),
        $"DP05_0005PE".as(per_under_5).cast("Double"),
        $"DP05_0006PE".as(per_5_9).cast("Double"),
        $"DP05_0007PE".as(per_10_14).cast("Double"),
        $"DP05_0008PE".as(per_15_19).cast("Double"),
        $"DP05_0009PE".as(per_20_24).cast("Double"),
        $"DP05_0010PE".as(per_25_34).cast("Double"),
        $"DP05_0011PE".as(per_35_44).cast("Double"),
        $"DP05_0012PE".as(per_45_54).cast("Double"),
        $"DP05_0013PE".as(per_55_59).cast("Double"),
        $"DP05_0014PE".as(per_60_64).cast("Double"),
        $"DP05_0015PE".as(per_65_74).cast("Double"),
        $"DP05_0016PE".as(per_75_84).cast("Double"),
        $"DP05_0017PE".as(per_85_plus).cast("Double")
    )

val complexRace17 = rawDem17.select(
        $"FIPS",
        $"Year".cast("Int"),
        $"DP05_0036PE".as(per_one_race).cast("Double"),
        $"DP05_0037PE".as(per_only_white).cast("Double"),
        $"DP05_0038PE".as(per_only_black).cast("Double"),
        $"DP05_0039PE".as(per_only_native).cast("Double"),
        $"DP05_0040PE".as(per_only_cherokee).cast("Double"),
        $"DP05_0041PE".as(per_only_chippewa).cast("Double"),
        $"DP05_0042PE".as(per_only_navajo).cast("Double"),
        $"DP05_0043PE".as(per_only_sioux).cast("Double"),
        $"DP05_0044PE".as(per_only_asian).cast("Double"),
        $"DP05_0045PE".as(per_only_indian).cast("Double"),
        $"DP05_0046PE".as(per_only_chinese).cast("Double"),
        $"DP05_0047PE".as(per_only_filipino).cast("Double"),
        $"DP05_0048PE".as(per_only_japanese).cast("Double"),
        $"DP05_0049PE".as(per_only_korean).cast("Double"),
        $"DP05_0050PE".as(per_only_vietnamese).cast("Double"),
        $"DP05_0051PE".as(per_only_other_asian).cast("Double"),
        $"DP05_0052PE".as(per_only_pacific_islander).cast("Double"),
        $"DP05_0053PE".as(per_only_hawaiian).cast("Double"),
        $"DP05_0054PE".as(per_only_guam).cast("Double"),
        $"DP05_0055PE".as(per_only_samoan).cast("Double"),
        $"DP05_0056PE".as(per_only_pacific_other).cast("Double"),
        $"DP05_0057PE".as(per_only_other).cast("Double"),
        $"DP05_0058PE".as(per_two_races).cast("Double"),
        $"DP05_0059PE".as(per_white_black).cast("Double"),
        $"DP05_0060PE".as(per_white_native).cast("Double"),
        $"DP05_0061PE".as(per_white_asian).cast("Double"),
        $"DP05_0062PE".as(per_black_native).cast("Double")
    )

val simpleRace17 = rawDem17.select(
        $"FIPS",
        $"Year".cast("Int"),
        $"DP05_0064PE".as(per_white).cast("Double"),
        $"DP05_0065PE".as(per_black).cast("Double"),
        $"DP05_0066PE".as(per_native).cast("Double"),
        $"DP05_0067PE".as(per_asian).cast("Double"),
        $"DP05_0068PE".as(per_pacific).cast("Double"),
        $"DP05_0069PE".as(per_other).cast("Double")
    )

val hispanic17 = rawDem17.select(
        $"FIPS",
        $"Year".cast("Int"),
        $"DP05_0071PE".as(per_hispanic).cast("Double"),
        $"DP05_0072PE".as(per_mexican).cast("Double"),
        $"DP05_0073PE".as(per_puerto_rican).cast("Double"),
        $"DP05_0074PE".as(per_cuban).cast("Double"),
        $"DP05_0075PE".as(per_hispanic_other).cast("Double"),
        $"DP05_0076PE".as(per_not_hispanic).cast("Double")
    )

genAge = genAge.union(genAge17)
complexRace = complexRace.union(complexRace17)
simpleRace = simpleRace.union(simpleRace17)
hispanic = hispanic.union(hispanic17)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rawDem17: org.apache.spark.sql.DataFrame = [GEO_ID: string, NAME: string ... 358 more fields]
res15: rawDem17.type = [GEO_ID: string, NAME: string ... 358 more fields]
genAge17: org.apache.spark.sql.DataFrame = [FIPS: string, Year: int ... 16 more fields]
complexRace17: org.apache.spark.sql.DataFrame = [FIPS: string, Year: int ... 27 more fields]
simpleRace17: org.apache.spark.sql.DataFrame = [FIPS: string, Year: int ... 6 more fields]
hispanic17: org.apache.spark.sql.DataFrame = [FIPS: string, Year: int ... 6 more fields]
genAge: org.apache.spark.sql.DataFrame = [FIPS: string, Year: int ... 16 more fields]
complexRace: org.apache.spark.sql.DataFrame = [FIPS: string, Year: int ... 27 more fields]
simpleRace: org.apache.spark.sql.DataFrame = [FIPS: string, Year: int ... 6 more fields]
hispanic: org.apache.spark.sql.DataFrame = [FIPS: string, Year: int ... 6 more fields]


In [5]:
// Test to make sure counts of all tables are the same, and no null values
val tables = List(genAge, complexRace, simpleRace, hispanic)

if (tables.exists(_.count != tables.head.count) || tables.exists(t => { t.count != t.na.drop.count } )) {
    println("Test failed")
} else { 
    println("Test passed")
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

tables: List[org.apache.spark.sql.DataFrame] = List([FIPS: string, Year: int ... 16 more fields], [FIPS: string, Year: int ... 27 more fields], [FIPS: string, Year: int ... 6 more fields], [FIPS: string, Year: int ... 6 more fields])
Test passed


In [6]:
for (table <- tables) {
    table.count
    table.printSchema
    table.orderBy(rand).show
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- FIPS: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Population: integer (nullable = true)
 |-- Male: double (nullable = true)
 |-- Female: double (nullable = true)
 |-- Under_5: double (nullable = true)
 |-- 5_9: double (nullable = true)
 |-- 10_14: double (nullable = true)
 |-- 15_19: double (nullable = true)
 |-- 20_24: double (nullable = true)
 |-- 25_34: double (nullable = true)
 |-- 35_44: double (nullable = true)
 |-- 45_54: double (nullable = true)
 |-- 55_59: double (nullable = true)
 |-- 60_64: double (nullable = true)
 |-- 65_74: double (nullable = true)
 |-- 75_84: double (nullable = true)
 |-- 85plus: double (nullable = true)

+-----+----+----------+----+------+-------+---+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+
| FIPS|Year|Population|Male|Female|Under_5|5_9|10_14|15_19|20_24|25_34|35_44|45_54|55_59|60_64|65_74|75_84|85plus|
+-----+----+----------+----+------+-------+---+-----+-----+-----+-----+-----+-----+-----+---

In [7]:
genAge.write.parquet("s3://agimodeltrainer/Clean_Data/Age_and_Gender.parquet")
complexRace.write.parquet("s3://agimodeltrainer/Clean_Data/Complex_Race.parquet")
simpleRace.write.parquet("s3://agimodeltrainer/Clean_Data/Simple_Race.parquet")
hispanic.write.parquet("s3://agimodeltrainer/Clean_Data/Hispanic.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
val tables = List("Age_and_Gender", "Complex_Race", "Simple_Race", "Hispanic")

for (t <- tables) {
    println(t)
    val test = spark.read.parquet(s"s3://agimodeltrainer/Clean_Data/${t}.parquet")
    println("Count: " + test.count)
    println("Count after drop: " + test.na.drop.count)
    test.printSchema
    test.orderBy(rand).show
}


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

tables: List[String] = List(Age_and_Gender, Complex_Race, Simple_Race, Hispanic)
Age_and_Gender
Count: 22543
Count after drop: 22543
root
 |-- FIPS: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Population: integer (nullable = true)
 |-- Male: double (nullable = true)
 |-- Female: double (nullable = true)
 |-- Under_5: double (nullable = true)
 |-- 5_9: double (nullable = true)
 |-- 10_14: double (nullable = true)
 |-- 15_19: double (nullable = true)
 |-- 20_24: double (nullable = true)
 |-- 25_34: double (nullable = true)
 |-- 35_44: double (nullable = true)
 |-- 45_54: double (nullable = true)
 |-- 55_59: double (nullable = true)
 |-- 60_64: double (nullable = true)
 |-- 65_74: double (nullable = true)
 |-- 75_84: double (nullable = true)
 |-- 85plus: double (nullable = true)

+-----+----+----------+----+------+-------+---+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+
| FIPS|Year|Population|Male|Female|Under_5|5_9|10_14|15_19|20_24|25_34|35_